# Weather Data Preprocessing 2# 


In this notebook we clean up the weather dataframe & merge it with the fire dataframe.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from google.colab import drive
drive.mount("/content/drive")
#pd.set_option('display.max_rows', None)

Mounted at /content/drive


- load the csv file that we got from Data_Preprocessing_Weather_1.ipynb into a dataframe

In [ ]:
weather_with_state = pd.read_csv('/content/drive/Shared drives/BNCS411_Final_Project/weather_with_state.csv', delimiter = ',')
fires_pred_knn = pd.read_csv('/content/drive/Shared drives/BNCS411_Final_Project/fires_pred_knn.csv', delimiter=',')
del fires_pred_knn['Unnamed: 0']

- drop all columns that we don't need as features later

In [ ]:
weather_with_state = weather_with_state.drop(['Unnamed: 0','Estacao','Hora', 'Precipitacao','TempBulboSeco', 'TempBulboUmido','UmidadeRelativa', 
                  'PressaoAtmEstacao', 'PressaoAtmMar', 'DirecaoVento','VelocidadeVento', 'Insolacao', 'Nebulosidade', 
                  'Evaporacao Piche','Temp Comp Media', 'Unnamed: 19'], 1)


- correct an error from Data_Preprocessing_Weather_1.ipynb (october 2000 needed to be dropped)

In [ ]:
weather_with_state['Data'] = pd.to_datetime(weather_with_state['Data'])
weather_with_state = weather_with_state[~weather_with_state['Data'].isin(pd.date_range(start='20001002', end='20001031'))]

- calculate the max/min-temperature, humidity & wind speed mean for each day per state

In [ ]:
weather_with_state.columns = ['Date', 'MaxTemp', 'MinTemp',	'RelHum', 'WindVel', 'State']
weather_with_state = weather_with_state.groupby(['Date', 'State']).mean()

- sort by state -> Date to allow interpolation from missing values. That way we can take the previous & next day to assume the missing value of that day.

In [ ]:
print(weather_with_state.isnull().sum())
weather_with_state = weather_with_state.sort_values(['State', 'Date'], ascending=True)
weather_with_state = weather_with_state.interpolate()
weather_with_state = weather_with_state.groupby(['Date', 'State']).mean()
print(weather_with_state.isnull().sum())

MaxTemp    0
MinTemp    0
RelHum     0
WindVel    0
dtype: int64

- rename the columns in fires_pred_knn to match the corresponding ones in weather_with_fires. 
- translate to datetime in order to make comparison possible.


In [ ]:
fires_pred_knn.columns = ['Date', 'State', 'Fires']
fires_pred_knn['Date'] = pd.to_datetime(fires_pred_knn['Date'])

- merge the two dataframes on the primary key pair Date+State

In [ ]:
weather_with_fires = fires_pred_knn.merge(weather_with_state,on = ['Date','State'], how = 'right')

- The days/states without fire did not get a row in fires_pred_knn therefore there are some rows that were merged with 'nothing' so there are some rows with NaN in the Fires column. As we know that there were no fires on those days, we can replace the NaN values with zeros.
- parse the Fires column to integer as the amount of fires Fires cannot have decimal values

In [ ]:
print(weather_with_fires.isnull().sum())
weather_with_fires.fillna(value=0, inplace=True)
weather_with_fires['Fires'] = weather_with_fires['Fires'].astype(int)

NameError: ignored

- save to .csv file

In [ ]:
weather_with_fires.to_csv('/content/drive/Shareddrives/BNCS411_Final_Project/weather_with_fires.csv')